In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import lxml
import time
import re

In [3]:
path = "C:/Users/user/chromedriver_win32_2/chromedriver.exe"
website= "https://www.obodo.be/fr/trouvez/Nettoyage+(Entreprises+de)/Provincie+Brussel/"

driver = webdriver.Chrome(path)
driver.get(website)
driver.maximize_window

<bound method WebDriver.maximize_window of <selenium.webdriver.chrome.webdriver.WebDriver (session="262cd2c637b8bccdbfc8904c1ff6e2f9")>>

In [22]:
box = driver.find_element_by_xpath("//div[@class= 'pos pos--rel']/ol")
items = box.find_elements_by_xpath(".//li[contains(@class, 'result-item')]")

In [88]:
def get_link(element):
 
    link = element.find_element_by_xpath('.//h2[@class="result-item__title"]/a').get_attribute('href')
    return link

In [176]:
box= driver.find_element_by_xpath('//ol[@class="result-items "]')
items= WebDriverWait(box, 6).until(EC.presence_of_all_elements_located((By.XPATH, 
                                                                      '//li[contains(@class, "result-item item-data ")]')))


links= []
scrolling = True
while True:
    for item in items:
        links.append(get_link(item))
        
        last_height= driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(7)
        new_height= driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            scrolling = False
            break
        else:
            last_height= new_height
            break 


In [80]:
#creating a function to reiterate over each link to get the relevant data we need
#if the link does not contain the data, it returns nan
def get_data(element):
    website= element
    data= requests.get(website)
    data= data.text
    soup= BeautifulSoup(data, 'lxml')
    box= soup.find('section',class_= 'profile-section')
    try:
        name= box.find('h1', class_= 'profile-section__title').get_text()
    except:
        name= np.nan
    try:
        ma= box.find_all('a', class_ = 'btn btn--action t-c', href=True)[1:]
        ma=str(ma)
        ma = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', ma)
        mail=ma.group(0)
        email= mail
    except:
        email= np.nan
    
    try:    
        street_address_numbers= box.find('span', class_= 'address__line').get_text().split(',')[0]
    except:
        street_address_numbers= np.nan
        
    try:
        zipcode = box.find('span', class_= 'address__line').get_text().split(',')[1].split(' ')[1]
    except:
        zipcode= np.nan
    
    try:    
        add= box.find('span', class_= 'address__line').get_text().split(',')[1].split(' ')[2:]
        address= ' '.join(add)
    except:
        address= np.nan
    
    try:
        phone = box.find('span', itemprop= 'telephone').get_text()
    except:
        phone= np.nan
    
    try:
        ma= box.find_all('a', class_ = 'btn btn--action t-c', href=True)[1:]
        ma=str(ma)
        ma = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', ma)
        mail=ma.group(0)
        email= mail
    except:
        email=np.nan
        
   
    data =  [name, email, street_address_numbers ,zipcode ,address , phone ,email]
    
    return data
    

In [181]:
#applying the above function to get the relevant data
#the formula will be applied to each link to get the name, email and so on
name= []
phone= []
email= []
street_address_number = []
zipcode=[]
address= []
for i in links[0:177]:     #scraping data from the first 177 links
    data= get_data(i)
    name.append(data[0])
    phone.append(data[5])
    email.append(data[1])
    street_address_number.append(data[2])
    zipcode.append(data[3])
    address.append(data[4])    

In [182]:
#creating a df with the data scraped
df0_177= pd.DataFrame({'name':name, 'phone': phone, 'email': email,
                           'street_address_number': street_address_number, 'zipcode':zipcode,
                           'address':address})

df0_177.shape

(177, 6)

In [138]:
df0_177.head()

,name,phone,email,street_address_number,zipcode,address
0,Sec O Net,081 73 86 34,info@seconet.be,Rue Patenier 7,5000,Namur
1,CED C'est NET,0496 30 48 75,cedcestnet@skynet.be,Rue Ferdinand Hendschel 30,5300,Seilles (Andenne)
2,NaN,NaN,NaN,NaN,NaN,NaN
3,Le coup de balai,071 68 71 30,info@lecoupdebalai.be,Rue de la Chapelle 11/A,5620,Florennes
4,Gestanet,081 30 33 36,info@gestanet.be,Rue de Dave 54,5100,Jambes (Namur)


In [10]:
df0_177.to_csv('links0_117.csv', index= False)

In [9]:
driver.quit()